<a href="https://colab.research.google.com/github/gnitnaw/Programs/blob/master/Chapter5/Hougen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.7.12


# Example 5.9
## Import modules

In [2]:
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
print(np.__version__)

1.19.5


## Set random seed

In [3]:
np.random.seed(5678)

## Original Program

In [4]:
#np.set_printoptions(precision=8)
Xmat = np.array([470.0, 300,  10,
                 285,  80,  10,  
                 470, 300, 120, 
                 470,  80, 120, 
                 470,  80,  10,
                 100, 190,  10,
                 100,  80,  65,
                 470, 190,  65,
                 100, 300,  54,
                 100, 300, 120,
                 100,  80, 120,
                 285, 300,  10,
                 285, 190, 120]).reshape(13,3)

Ymat = np.array([8.55, 3.79, 4.82, 0.02, 2.75, 14.39, 2.54,
                 4.35, 13.00, 8.50, 0.05, 11.32, 3.13])
beta_D = 5  # dim of beta
n, D = Xmat.shape  # n: No. of sample, D: dim. of X
N = 10000   # Generate sample number
rho = 0.05  # elite sample rate
rho2 = 0.5

def HougenFunc(x, beta):
    return (beta[0]*x[1]-x[2]/beta[4])/(1+beta[1]*x[0]+beta[2]*x[1]+beta[3]*x[2])

def lossFunc(X, Y, beta, n):
    err = 0.0
    for i in range(n):
        err += (Y[i]-HougenFunc(X[i], beta))**2
        #print(X[i])
        #print(HougenFunc(X[i], beta))
        #print(Y[i])
        #print(err)

    return err/13

def minLoss(N, n, D, X, Y):
    Ne = int(rho*N)
    beta_cand = np.random.uniform(0.0, 1.0, (N, beta_D))
    errs = np.zeros(N)
    #print(beta_cand)
    for k in range(100):
        for i in range(N):
            errs[i] = lossFunc(X, Y, beta_cand[i], n)
        #print(errs)
        sortErrs = np.argsort(errs)
        err = errs[sortErrs[0]]
        elIds = sortErrs[0:Ne]
        elites = beta_cand[elIds].copy()
        means = np.mean(elites, axis=0)
        stds = np.std(elites, axis=0)
        #print(elites)
        beta_temp = np.zeros((beta_D, N))
        for i in range(beta_D):
            beta_temp[i] = np.random.normal(means[i], stds[i], N)
        beta_cand = beta_temp.T
        for i in range(Ne):
            beta_cand[i] = elites[i]

    return beta_cand[0], err

def minLossBeta(beta, N, n, D, X, Y):
    Ne = int(rho*N)
    beta_temp = np.zeros((beta_D, N))
    for i in range(beta_D):
        beta_temp[i] = np.random.normal(beta[i], beta[i]*0.05, N)
    beta_cand = beta_temp.T
    beta_cand[0] = beta
    errs = np.zeros(N)
    #print(beta_cand)
    for k in range(100):
        for i in range(N):
            errs[i] = lossFunc(X, Y, beta_cand[i], n)
        #print(errs)
        sortErrs = np.argsort(errs)
        err = errs[sortErrs[0]]
        elIds = sortErrs[0:Ne]
        elites = beta_cand[elIds].copy()
        means = np.mean(elites, axis=0)
        stds = np.std(elites, axis=0)
        #print(elites)
        beta_temp = np.zeros((beta_D, N))
        for i in range(beta_D):
            beta_temp[i] = np.random.normal(means[i], stds[i], N)
        beta_cand = beta_temp.T
        for i in range(Ne):
            beta_cand[i] = elites[i]

    return beta_cand[0], err

beta_test = np.array([1.2526, 0.0628, 0.0400, 0.1124, 1.1914])

beta_Best, err = minLoss(N, n, D, Xmat, Ymat)
bb, ee = minLossBeta(beta_Best, N, n, D, Xmat, Ymat)
print("beta = ", bb, " , err = ", ee)
print("beta of book = ", beta_test, ", err = ", lossFunc(Xmat, Ymat, beta_test, n))



beta =  [1.23237211 0.06176051 0.03932972 0.11066961 1.21116812]  , err =  0.022994030525393647
beta of book =  [1.2526 0.0628 0.04   0.1124 1.1914] , err =  0.023000681815053
